<b>Betweenness Centrality for Removing Nodes, part 3</b>

This notebook is for reviewing how to find the betweenness centrality for nodes and finding how to remove individual nodes from graphs, and how that affects the set.

Part 2 reviews removing edges versus nodes.

In [59]:
import csv

import numpy as np
import networkx as nx 
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
#Select some data to work with for this session

#first create list using all columns
allData = []
with open('one.csv', 'r') as f:
    filereader = csv.reader(f, delimiter="\t", quotechar='"')
    #next(filereader) # skips header row, we don't want to skip since we don't have a header
    for row in filereader:
            allData.append(row)
            
            
#for this practice, lets keep seperate some data by dates and we can compare them!
#All the interactions during the 1840's using a new bit of code
forties = []
edge = []
i=0
for x in allData:
    date=x[2].split("/")
    if len(date)==3:
        if date[2].startswith('4'): #This helps single out decades instead of specific years
        #if date[2]=="49":
            edge.append([x[3],x[4], x[2]])
            forties.append(edge[i])
            i=i+1
            
#fix the solver column by seperating out responders, whitespace, and formatting unknowns
fixedSolver = []
edge = []
i=0

unknown = {"", "Unknown", "unknown"}
for x in forties:
    if ";" in x[1]:
        solvers = x[1].split(";")
        for y in solvers:
            one=x[0].replace(' ', '')
            two=y.replace(' ', '')
            if(one in unknown):
                one="UNKNOWN"
            if(two in unknown):
                two="UNKNOWN"
            fixedSolver.append([one,two])
    else:
        one=x[0].replace(' ', '')
        two=x[1].replace(' ', '')
        if(one in unknown):
            one="UNKNOWN"
        if(two in unknown):
            two="UNKNOWN"
        fixedSolver.append([one,two])
    #i=i+1

<b>Begin trying to select edges to remove</b>

In [61]:
#find the edges with the highest centrality
from networkx import edge_betweenness_centrality
import operator
def my_central_edge(G):
    centrality = edge_betweenness_centrality(G)
    sortedCent = sorted(centrality.items(), key=operator.itemgetter(1), reverse=True)
    edge = sortedCent[0]
    return edge[0]

In [56]:
#make new communities by removing one edge at a time
from networkx import number_connected_components
def make_community(edgeList, iterations):
    
    tempList = list(edgeList)
    graph = nx.from_edgelist(edgeList)
    tempG = nx.from_edgelist(tempList)
    
    communities = number_connected_components(graph)
    compCommunities = number_connected_components(tempG)

    while compCommunities < communities + iterations:
        edge = my_central_edge(tempG)
        for x in tempList:
            if (x[0] == edge[0] and x[1] == edge[1]) or (x[0] == edge[1] and x[1] == edge[0]):
                print(x)
                tempList.remove(x)
            tempG = nx.from_edgelist(tempList)
            compCommunities = number_connected_components(tempG)
    #return tempList

In [58]:
#tested previously, now to start upping the interations to see how it handles equal centralities
colorList = make_community(fixedSolver, 1)
#colormap is a "generator of sets", contains dictionaries in this case
from networkx import connected_components
colorMap = nx.from_edgelist(colorList)
#add each dictionary to a list so they're easier to pick from
nodes = []
[nodes.append(c) for c in sorted(nx.connected_components(colorMap), key=len, reverse=True)]
#node create a list of lists with nodes in each
#final list will have lists containing nodes in a single component
nodeList = []
i=0;
for x in nodes:
    nodeList.append([])
    for y in x:
        nodeList[i].append(y)
    i+=1
nodeList

['ThomasWilkinson', 'J.W.']
['ThomasWilkinson', 'ThomasWilkinson*']
['Geometricus', 'UNKNOWN']
['Geometricus', 'UNKNOWN']


TypeError: 'NoneType' object is not iterable

In [ ]:
graph = nx.from_edgelist(fixedSolver)
centrality = edge_betweenness_centrality(G)
sortedCent = sorted(centrality.items(), key=operator.itemgetter(1), reverse=True)